## run_inference Fonksiyonunda Ben Aslında Ne Yapıyorum?

```cpp
torch::Tensor run_inference(torch::jit::script::Module &module,
                            const torch::Tensor &input)
{
    // TorchScript forward() IValue listesi bekler
    std::vector<torch::jit::IValue> inputs;
    inputs.push_back(input);

    // module.forward() -> IValue, toTensor() ile tensöre çevir
    torch::Tensor output = module.forward(inputs).toTensor();
    return output;
}
```


---
# 1) Fonksiyonun amacı

Bu fonksiyonun olayı şu:

**“Ben bu fonksiyonla, hazır yüklenmiş TorchScript modelime (module) bir giriş tensörü (input) verip, modelden çıkan sonucu tensör olarak geri alıyorum.”**

Yani fonksiyon benim için tek iş yapan temiz bir kutu:

* İçine: model + input tensör

* Dışına: output tensör

#  2) Parametreler – Neden böyle tanımladım?
```cpp
torch::jit::script::Module &module,
const torch::Tensor &input
```


### Module &module
* Burada ben modülü referans ile alıyorum:

* Kopya oluşmasını istemiyorum (model kopyalamak pahalı).

* Zaten load ile dışarıda bir kere yükledim, burada sadece onu kullanıyorum.

### const torch::Tensor &input
Burada da:

* Input tensörün kopyasını almak istemiyorum.

* Fonksiyon içinde input’u değiştirme niyetim yok.

* O yüzden const & ile “okuma amaçlı, kopyasız” kullanıyorum.

## Eğer:

* Değerle (Module module) alsaydım → her çağrıda model kopyalanırdı, çok gereksiz ve maliyetli olurdu.

* input’u kopyalayarak alsaydım → her çağrıda ekstra RAM ve hız kaybı yaşardım.

# 3) std::vector<torch::jit::IValue> inputs; – Neden IValue?
```cpp
std::vector<torch::jit::IValue> inputs;
inputs.push_back(input);
```


* TorchScript forward fonksiyonu düz tensör beklemiyor, 
>std::vector<torch::jit::IValue> bekliyor.

### Ben burada şunu yapıyorum:

* Bir inputs listesi oluşturuyorum.

### inputs.push_back(input); dediğimde:

* torch::Tensor otomatik olarak IValue içine sarılıyor.

* Yani tensörü “TorchScript’in genel değer tipine” çeviriyorum.

## **Neden liste?**

**Çünkü TorchScript modeli teorik olarak birden fazla giriş alabilir:**

* Örneğin (image, extra_features) gibi.

**Benim modelim tek input alıyor ama API genel olduğu için ben yine de vektör kullanmak zorundayım**.

# 4)module.forward(inputs).toTensor() – Burada ne oluyor?
```cpp
torch::Tensor output = module.forward(inputs).toTensor();
```


Burada zincir şöyle çalışıyor:

### module.forward(inputs)

* Ben burada TorchScript modeline forward çağrısı yapıyorum.

* Dışarıdan verdiğim inputs listesindeki IValue’ler modele input oluyor.

* Geriye yine bir torch::jit::IValue dönüyor (çıktının genel temsili).

### .toTensor()

* Ben bu IValue’nin aslında tensör olduğunu biliyorum.

* O yüzden IValue’yi toTensor() ile tekrar tensöre çeviriyorum.

>Sonuç: torch::Tensor output

### Eğer -> Modelim aslında tensör değil de tuple, dict vs. döndürseydi:

* toTensor() çağırdığım yerde runtime hatası alırdım.

### Yanlış input shape verirsem:

* Ya forward içinde dimension hatası alırım,

* Ya da model saçma çıktı üretir ama yine tensör döner.

# 5) Bu fonksiyonu böyle yazmasam ne sıkıntı yaşardım?

### 1.)Module’ü referans yerine kopya alsam

* Her fonksiyon çağrısında model kopyalanır.

* Bu hem yavaş hem saçma, özellikle büyük modellerde ciddi performans kaybı.

### 2.)Input’u kopyalayarak alsam (const ref yerine)

* Her defasında input tensör RAM’de çoğalır.

* Büyük görüntülerde gereksiz hafıza ve zaman kaybı.

### 3.)IValue vektörü kullanmasam

* forward derlenmez, API uymaz.

### 4.)toTensor() kullanmasam

* Çıkan IValue ile uğraşmam gerekir, her yerde saçma dönüşümler yapmak zorunda kalırım.

* Ben çıktı ile tensör seviyesinde çalışmak istediğim için bu kullanım en temiz hali.

# 6) Ben bu fonksiyonu şöyle özetliyorum

**“Ben run_inference fonksiyonunda, TorchScript modelimi kopyalamadan referans olarak alıyorum, input tensörümü IValue listesine sarıp forward’a veriyorum, sonra da dönen IValue’yi tekrar tensöre çevirip çıktıyı tek bir yerden yönetilebilir hale getiriyorum.”**

----------

---
----
----

# Şimdi sıra main fonksiyonunda.Unutmayın biz bu açıklamaları şu ana kadar en kapsamlı olan Uygulama - 5 için yapıyoruz.Eğer ilk başta bu .ipynb dosyasını inceliyorsanız , uygulamaları takip etmenizi öneririm.Genel olarak uygulama içerisinde bu adımların açıklamaları mevcuttur.

----
---
---
---

# Main Fonksiyonu ( int main () )
```cpp

int main()
{
    try
    {
        std::cout << "[INFO] Program basladi.\n";

        const std::string model_path = "model_ts.pt";

        {
            std::ifstream f(model_path);
            if (!f.good())
            {
                std::cerr << "[HATA] model_ts.pt bulunamadi!\n";
                std::cerr << "       model_ts.pt dosyasini exe ile ayni klasore koy.\n";
                std::cout << "Enter'a basip cikabilirsin...\n";
                std::cin.get();
                return -1;
            }
        }


        std::cout << "[INFO] Model yukleniyor...\n";
        torch::jit::script::Module module = torch::jit::load(model_path);

        // CPU kullanıyoruz (GPU varsa kCUDA da kullanılabilir)
        module.to(torch::kCPU);
        module.eval();
        std::cout << "[OK] Model yuklendi ve eval modunda.\n";


        const std::string image_path = "test.jpg";
        cv::Mat img_bgr = cv::imread(image_path);

        if (img_bgr.empty())
        {
            std::cerr << "[HATA] Resim yuklenemedi: " << image_path << "\n";
            std::cerr << "       test.jpg dosyasini exe ile ayni klasore koy.\n";
            std::cout << "Enter'a basip cikabilirsin...\n";
            std::cin.get();
            return -1;
        }

        std::cout << "[OK] Resim yuklendi. Boyut: "
                  << img_bgr.cols << "x" << img_bgr.rows << " (W x H)\n";

        int target_size = 64; 
        torch::Tensor input_tensor = preprocess_image(img_bgr, target_size);

        std::cout << "[DEBUG] Input tensor shape: " << input_tensor.sizes() << "\n";

        // Beklenen shape: [1, 3, 64, 64]
        auto sizes = input_tensor.sizes();
        if (!(sizes.size() == 4 &&
              sizes[0] == 1 &&
              sizes[1] == 3 &&
              sizes[2] == target_size &&
              sizes[3] == target_size))
        {
            std::cerr << "[HATA] Input tensor beklenenden farkli! Gelen: "
                      << sizes << "\n";
            std::cout << "Enter'a basip cikabilirsin...\n";
            std::cin.get();
            return -1;
        }


        std::cout << "[INFO] Inference calistiriliyor...\n";
        torch::Tensor output = run_inference(module, input_tensor);

        std::cout << "[DEBUG] Output tensor shape: " << output.sizes() << "\n";

        torch::Tensor probs = torch::softmax(output, 1); 
        torch::Tensor pred_class = probs.argmax(1);      

        std::cout << "Output (logits): " << output << "\n";
        std::cout << "Predicted class index: "
                  << pred_class.item<int>() << "\n";

        std::cout << "[INFO] Program bitti. Enter'a basip cikabilirsin...\n";
        std::cin.get();
    }
    catch (const c10::Error &e)
    {
        std::cerr << "[EXCEPTION - c10] " << e.what() << "\n";
        std::cout << "Enter'a basip cikabilirsin...\n";
        std::cin.get();
        return -1;
    }
    catch (const std::exception &e)
    {
        std::cerr << "[EXCEPTION - std] " << e.what() << "\n";
        std::cout << "Enter'a basip cikabilirsin...\n";
        std::cin.get();
        return -1;
    }

    return 0;
}
```
----

## 1️⃣ try–catch yapısı ve giriş logları
```cpp
int main()
{
    try
    {
        std::cout << "[INFO] Program basladi.\n";
        ...
    }
    catch (const c10::Error &e)
    {
        ...
    }
    catch (const std::exception &e)
    {
        ...
    }

    return 0;
}
```


Burada ben şunu yapıyorum:

* Tüm main akışını try içine alıyorum.

- Eğer LibTorch tarafında bir hata olursa c10::Error yakalıyorum.

- Normal C++ istisnaları için std::exception yakalıyorum.

- Hata olduğunda mesajı basıp std::cin.get() ile program hemen kapanmasın diye kullanıcıdan Enter bekliyorum.

Yani:

**“Ben main’i try–catch içine alarak, model yükleme / inference sırasında patlayan hataları kontrollü yakalıyorum ve pencereyi hemen kapatmak yerine kullanıcıya hata mesajını okutuyorum.”**

## 2️⃣ Model yolunu belirleme ve dosya kontrolü

```cpp
const std::string model_path = "model_ts.pt";

{
    std::ifstream f(model_path);
    if (!f.good())
    {
        std::cerr << "[HATA] model_ts.pt bulunamadi!\n";
        std::cerr << "       model_ts.pt dosyasini exe ile ayni klasore koy.\n";
        std::cout << "Enter'a basip cikabilirsin...\n";
        std::cin.get();
        return -1;
    }
}
```

Burada ben:

* model_path ile TorchScript model dosyamın adını sabitliyorum.

* std::ifstream ile dosya gerçekten var mı diye kontrol ediyorum.

Dosya yoksa:

* LibTorch tarafında karmaşık bir hata yerine,

* Kullanıcıya net bir mesaj gösteriyorum ve programı temizce sonlandırıyorum.

Yani:

**“Ben model dosyasının varlığını LibTorch’a bırakmak yerine kendim kontrol ediyorum ki, kullanıcı insanca bir hata mesajı görsün.”**


## 3️⃣ TorchScript modelini yükleme ve hazırlama
```cpp
std::cout << "[INFO] Model yukleniyor...\n";
torch::jit::script::Module module = torch::jit::load(model_path);

// CPU kullanıyoruz (GPU varsa kCUDA da kullanılabilir)
module.to(torch::kCPU);
module.eval();
std::cout << "[OK] Model yuklendi ve eval modunda.\n";
```

Burada :

* torch::jit::load ile .pt dosyasından TorchScript modeli yüklüyorum.

* module.to(torch::kCPU); ile modeli CPU üzerine taşıyorum.

### module.eval(); ile modeli evaluation moduna alıyorum:

* Dropout vs. gibi eğitimde davranan katmanlar inference moduna geçiyor.

* Log’larla da adımları takip edilebilir hale getiriyorum.

### Yapmazsam ne olur?

#### eval() yapmazsam:

* Eğitim davranışı devam eder, özellikle dropout/BatchNorm gibi katmanlar tutarsız sonuç verebilir.

#### to(torch::kCPU) koymazsam:

*  Varsayılan genelde CPU olur ama ileride GPU kullanırsam karışıklık çıkar; explicit yazmak kontrolü bana veriyor.

## 4️⃣ Test görselini okuma
```cpp
const std::string image_path = "test.jpg";
cv::Mat img_bgr = cv::imread(image_path);

if (img_bgr.empty())
{
    std::cerr << "[HATA] Resim yuklenemedi: " << image_path << "\n";
    std::cerr << "       test.jpg dosyasini exe ile ayni klasore koy.\n";
    std::cout << "Enter'a basip cikabilirsin...\n";
    std::cin.get();
    return -1;
}

std::cout << "[OK] Resim yuklendi. Boyut: "
          << img_bgr.cols << "x" << img_bgr.rows << " (W x H)\n";
```

Burada :

* cv::imread ile resmi BGR formatında yüklüyorum.

* img_bgr.empty() ile dosya gerçekten okunmuş mı kontrol ediyorum.

* Hata durumunda yine kullanıcıya düzgün mesaj verip çıkıyorum.

* Başarılıysa, resmin genişlik–yükseklik bilgisini logluyorum.

## 5️⃣ Preprocess çağrısı ve input tensör shape kontrolü
```cpp
int target_size = 64; // Python tarafında Resize((64,64)) kullandık
torch::Tensor input_tensor = preprocess_image(img_bgr, target_size);

std::cout << "[DEBUG] Input tensor shape: " << input_tensor.sizes() << "\n";

auto sizes = input_tensor.sizes();
if (!(sizes.size() == 4 &&
      sizes[0] == 1 &&
      sizes[1] == 3 &&
      sizes[2] == target_size &&
      sizes[3] == target_size))
{
    std::cerr << "[HATA] Input tensor beklenenden farkli! Gelen: "
              << sizes << "\n";
    std::cout << "Enter'a basip cikabilirsin...\n";
    std::cin.get();
    return -1;
}
```
Burada :

### preprocess_image ile:

* Resize

* BGR → RGB

* Normalize

* cv::Mat → torch::Tensor

işlemlerini tek fonksiyonda topluyorum.

**Sonra tensor shape’ini sizes() ile alıp kontrol ediyorum:**

* 4 boyutlu mu?

* Batch = 1 mi?

* Kanal = 3 mü?

* Yükseklik/genişlik = 64 mü?

#### Bunu yapmamın sebebi:

* “Ben input tensörün modelle birebir uyumlu olduğunu garanti altına alıyorum. Aksi halde debug çok zorlaşır.”

### Yapmazsam:

* Yanlış shape modelin içinde patlar.

* Hata mesajı daha karışık olur (conv içinde dimension mismatch vs.).

## 6️⃣ Inference çalıştırma
```cpp
std::cout << "[INFO] Inference calistiriliyor...\n";
torch::Tensor output = run_inference(module, input_tensor);

std::cout << "[DEBUG] Output tensor shape: " << output.sizes() << "\n";
```

Burada:

* run_inference fonksiyonunu çağırarak modeli input tensörle çalıştırıyorum.

* Çıkan tensörün shape’ini debug için yazdırıyorum.

* Genelde [1, num_classes] bekliyorum (örneğin [1, 10]).

### Bu ayrı fonksiyon yapısının bana faydası:

**“Ben forward çağrısını main’den soyutladım. Böylece inference akışını ayrı bir fonksiyon içinde temizce yönetebiliyorum.”**

## 7️⃣ Postprocess: softmax + argmax + item
```cpp
torch::Tensor probs = torch::softmax(output, 1); // sınıf olasılıkları
torch::Tensor pred_class = probs.argmax(1);      // en yüksek olasılıklı sınıf index'i

std::cout << "Output (logits): " << output << "\n";
std::cout << "Predicted class index: "
          << pred_class.item<int>() << "\n";
```

Burada :

* output tensörünü softmax ile olasılığa çeviriyorum.

* argmax(1) ile en yüksek olasılıklı sınıfın indeksini buluyorum.

* itemint>() ile bu index’i C++ tarafında normal bir int olarak alıyorum.

* Hem ham logits değerlerini hem de tahmin edilen sınıf indeksini logluyorum.

### Yapmazsam:

**Softmax kullanmadan da argmax yapabilirim; ama:**

* Olasılık dağılımını görmek istesem softmax’a ihtiyaç var.

* Eğitim–inference tutarlılığı açısından softmax mantıklı.

## 8️⃣ Program sonu: Enter bekleme
```cpp
std::cout << "[INFO] Program bitti. Enter'a basip cikabilirsin...\n";
std::cin.get();
```

Burada da:

**“Ben program bittiğinde konsol penceresinin anında kapanmaması için kullanıcıdan Enter bekliyorum, böylece logları rahatça görebiliyorum.”**

## 9️⃣ catch blokları – Hata yakalama stratejisi
```cpp
catch (const c10::Error &e)
{
    std::cerr << "[EXCEPTION - c10] " << e.what() << "\n";
    ...
}
catch (const std::exception &e)
{
    std::cerr << "[EXCEPTION - std] " << e.what() << "\n";
    ...
}
```

Burada:

* LibTorch kaynaklı hatalar için c10::Error ayrı tutuluyor.

* Diğer standart C++ hataları için std::exception.

* İkisinde de kullanıcıya mesaj gösterip Enter bekliyorum.

Bu sayede:

**“Ben hem LibTorch hem de genel C++ hatalarını ayrı ayrı kategorize edebiliyorum ve debug ederken hangi tarağın patladığını net görebiliyorum.”**



-----